# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [1]:
# YOUR CHANGES HERE
#Blackboard material released: 01/16/26; Include to 12/2025
#imports
import yfinance as yf
import pandas as pd
import numpy as np
#define tickers and start/end dates
tickers = ["AAPL", "AMZN", "MSFT", "RBLX", "SPY", "TSLA"]
start_date = "2024-01-01"
end_date = "2025-12-31"
#download monthly data from Yahoo Finance
data = yf.download(
    tickers,
    start=start_date,
    end=end_date,
    interval="1mo", #1 month intervals 
    auto_adjust=False,
    progress=False
)
#pull adjusted close prices
adj_close = data["Adj Close"]
#reshape into cleaner format
monthly_prices = (
    adj_close
    .reset_index()
    .melt(id_vars="Date", var_name="Ticker", value_name="Adj Close")
    .sort_values(["Ticker","Date"])
    .reset_index(drop=True)
)
#make sure each ticker has exactly 24mos
monthly_prices.groupby("Ticker").count()

,Date,Adj Close
Ticker,,
AAPL,24,24
AMZN,24,24
MSFT,24,24
RBLX,24,24
SPY,24,24
TSLA,24,24


Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


In [2]:
# YOUR CHANGES HERE
#adjust column names so they fit format
adj_close = adj_close.reset_index()
adj_close = adj_close.rename(columns={"Date": "date"})
#adjust to year-month-date format
adj_close["date"] = adj_close["date"].dt.to_period("M").dt.to_timestamp("M")
#save as tsv file:
adj_close.to_csv(
    "historical_prices.tsv",
    sep="\t",
    index=False
)

Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [3]:
# YOUR CHANGES HERE
#use pandas .pct_change()
returns = adj_close.copy()
#set date as index
returns = returns.set_index("date")
#calculate monthly returns
returns = returns.pct_change()
#drop first row (NaN returns b/c no dec. 2023 data included)
returns = returns.dropna()
returns.head(3)

Ticker,AAPL,AMZN,MSFT,RBLX,SPY,TSLA
date,,,,,,
2024-02-29,-0.019794,0.138918,0.040394,0.028086,0.052187,0.077901
2024-03-31,-0.050076,0.020480,0.018997,-0.043108,0.029503,-0.129235
2024-04-30,-0.006706,-0.029826,-0.074610,-0.068622,-0.037338,0.042608


Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [4]:
# YOUR CHANGES HERE
returns.reset_index().to_csv(
    "historical_returns.tsv",
    sep="\t",
    index=False
)

Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [5]:
# YOUR CHANGES HERE
expected_returns = returns.mean() #pd.series
#convert to df 
er_df = pd.DataFrame({
    "asset": expected_returns.index,
    "estimated_return": expected_returns.values
})

Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [6]:
# YOUR CHANGES HERE
er_df.reset_index().to_csv(
    "estimated_returns.tsv",
    sep="\t",
    index=False
)

Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [7]:
# YOUR CHANGES HERE
cov_matrix = returns.cov() #make covariance matrix for asset returns
cov_matrix.head(3)

Ticker,AAPL,AMZN,MSFT,RBLX,SPY,TSLA
Ticker,,,,,,
AAPL,0.003476,0.000104,0.000068,-0.001786,0.000542,0.002559
AMZN,0.000104,0.005046,0.002655,0.003803,0.001496,0.004629
MSFT,0.000068,0.002655,0.003555,0.004604,0.001292,0.002905


Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [8]:
# YOUR CHANGES HERE
#save to tsv w requirements listed above
cov_tsv = cov_matrix.reset_index() #asset names as first col.
cov_tsv = cov_tsv.rename(columns={cov_tsv.columns[0]: ""}) #rename first col so it's blank
cov_tsv.to_csv(
    "estimated_covariance.tsv",
    sep="\t",
    index=False
)

Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [9]:
# YOUR CHANGES HERE
#convert expected returns to numpy array
er = expected_returns.values
#find the asset w/ highest expected return
max_idx = np.argmax(er)

#build portfolio weights
weights = np.zeros_like(er)
weights[max_idx] = 1.0

#put portfolio results in df
portfolio = pd.DataFrame({
    "Ticker": expected_returns.index,
    "Weight": weights
})

Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [10]:
# YOUR CHANGES HERE
#redefine column names
portfolio = portfolio.rename(columns={"Ticker": "asset", "Weight":"allocation"})
#save as tsv
portfolio.to_csv(
    "maximum_return.tsv",
    sep="\t",
    index=False
)

Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum risk portfolio based on your previously estimated risks.

In [11]:
# YOUR CHANGES HERE
import cvxpy as cp #mentioned in lecture notes
#covariance matrix -> numpy
cov = cov_matrix.values
n = cov.shape[0]

#portfolio weights as cvxpy variable
w = cp.Variable(n)
#portfolio var.
portfolio_var = cp.quad_form(w, cov)
#constraints: weights sum to 1, weights >= 0
constraints = [cp.sum(w) == 1, w >= 0]

#define problem -> minimize var.
prob = cp.Problem(cp.Minimize(portfolio_var), constraints)
prob.solve()

#get weights
weights = w.value
weights_clean = np.clip(weights, 0, 1) #replaces small negative #s with 0

#put results in df
min_risk_portfolio = pd.DataFrame({
    "asset": cov_matrix.columns,
    "allocation": weights_clean
})

Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [12]:
# YOUR CHANGES HERE
min_risk_portfolio.to_csv(
    "minimum_risk.tsv",
    sep="\t",
    index=False
)

Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [13]:
# YOUR CHANGES HERE
tickers = cov_matrix.columns
n = len(tickers)

#calc. min and max returns
min_risk_return = (weights_clean @ expected_returns.values)
max_return = (weights @ expected_returns.values)
#target returns
tar_returns = np.linspace(min_risk_return, max_return, 101)

#efficient frontier portfolios
efficient_portfolios = []

for r in tar_returns:
    w = cp.Variable(n)
    port_var = cp.quad_form(w, cov_matrix.values)
    cs = [
        cp.sum(w) == 1,
        w >= 0,
        w @ expected_returns.values == r
    ]
    p = cp.Problem(cp.Minimize(port_var), cs)
    p.solve()
    efficient_portfolios.append(w.value)

#convert results to df
efficient_df = pd.DataFrame(efficient_portfolios, columns=tickers)

#compute portfolio volatilities (row-wise)
vols = []
for w in efficient_df.values:
    vols.append(np.sqrt(w.T @ cov_matrix.values @ w))
efficient_df.insert(0, "risk", vols)  #add volatility col first
efficient_df.insert(0, "return", tar_returns)  #add return col before risk
efficient_df.insert(0, "index", range(101))  #add index col

efficient_df.head(3)


Ticker,index,return,risk,AAPL,AMZN,MSFT,RBLX,SPY,TSLA
0,0,0.016857,0.030312,0.128449,-7.955806e-22,2.793970e-21,-9.860431e-21,0.871551,-1.280487e-20
1,1,0.016857,0.030312,0.128449,-7.955806e-22,2.793970e-21,-9.860431e-21,0.871551,-1.280487e-20
2,2,0.016857,0.030312,0.128449,-7.955806e-22,2.793970e-21,-9.860431e-21,0.871551,-1.280487e-20


Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [14]:
# YOUR CHANGES HERE
efficient_df.to_csv(
    "efficient_frontier.tsv", 
    sep="\t", 
    index=False
    )

Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [15]:
# YOUR CHANGES HERE
#set up arrays
n_sim = 1000 #number of simulations
n_samples = 23 #nymber of months in each resample
#store which asset is max return in each resample
max_return_counts = pd.Series(0, index=expected_returns.index)

#resample
for i in range(n_sim):
    #gen. 23 monthly return samples from multivar. normal
    resampled_returns = np.random.multivariate_normal(
        mean=expected_returns.values,
        cov=cov_matrix.values,
        size=n_samples
    )
    #compute mean return of each asset from the resample
    resampled_means = resampled_returns.mean(axis=0)
    #find asset w/highest return
    max_idx = np.argmax(resampled_means)
    #increment count for that asset
    max_return_counts.iloc[max_idx] += 1

#convert counts to probabilities 
max_return_prob = max_return_counts / n_sim

#put results in df
max_return_prob_df = pd.DataFrame({
    "asset": max_return_prob.index,
    "probability": max_return_prob.values
})

Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [16]:
# YOUR CHANGES HERE
max_return_prob_df.to_csv(
    "max_return_probabilities.tsv",
    sep='\t',
    index=False
)

Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgments are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

In [17]:
file_content = "For this assignment, I referenced the documentation on yfinance and cvxpy. Both of these were mentioned in the lecture notes, but I found it helpful to look at the documentation to better understand how to use it"
with open('acknowledgements.txt', 'w') as f:
    f.write(file_content)

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.